In [86]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, explode, split, count, col
from pyspark.sql.types import StructType, StructField, StringType

#необходимая библиотека для интеграции Spark и Postgres
spark_jars_packages = ",".join(
        [
            "org.postgresql:postgresql:42.4.0",
        ]
    )

In [17]:
#создаём SparkSession и передаём библиотеку для работы с Postgres
spark = SparkSession.builder \
    .appName("join data") \
    .config("spark.jars.packages", spark_jars_packages) \
    .getOrCreate()

In [18]:
#вычитываем данные из таблицы
tableDF = spark.read \
                    .format('jdbc') \
                    .option('url', 'jdbc:postgresql://rc1a-fswjkpli01zafgjm.mdb.yandexcloud.net:6432/de') \
                    .option('driver', 'org.postgresql.Driver') \
                    .option('dbtable', 'words') \
                    .option('user', 'student') \
                    .option('password', 'de-student') \
                    .load()

In [64]:
tableDF.show()

+---+---------+
| id|    words|
+---+---------+
|  1|    spark|
|  2|      sql|
|  3|     core|
|  4|streaming|
+---+---------+



In [19]:
#определяем схему для датафрейма
userSchema = StructType([StructField("text", StringType(), True)])

In [28]:
#читаем текст из файла
wordsDF = spark.readStream.schema(userSchema).format('text').load('/datas8')

In [53]:
splitWordsDF = wordsDF \
    .select(explode(split('text', ',')).alias('words') )

In [63]:
wordCountsDF = splitWordsDF \
    .groupBy('words') \
    .count()

In [62]:
wordCountsDF.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() \
    .awaitTermination() 

23/01/28 17:47:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-953583b4-b365-4b85-8274-cf9055ffe89d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/28 17:47:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+-----+
|         words|count|
+--------------+-----+
|spark context.|    1|
|          core|    1|
|     streaming|    2|
|           rdd|    1|
|           sql|    1|
| spark session|    1|
|       dataset|    1|
|         spark|    4|
|       pyspark|    1|
|    structured|    1|
|     dataframe|    1|
+--------------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [94]:
joinDF = wordCountsDF \
    .join(tableDF, ['words'], 'left') \
    .filter(col('id').isNull()) \
    .select('words')


In [95]:
joinDF.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() \
    .awaitTermination() 

23/01/28 18:12:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2d7d38f5-56b7-4a27-b0d5-fb53bdfc63a0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/28 18:12:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+
|         words|
+--------------+
|spark context.|
|           rdd|
| spark session|
|       dataset|
|       pyspark|
|    structured|
|     dataframe|
+--------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 